## Point clouds using GNPL

In [1]:

# !pip install -q torch torchvision torchaudio transformers numpy open3d pillow matplotlib

import os
import glob
import numpy as np
from PIL import Image

import torch
from transformers import GLPNImageProcessor, GLPNForDepthEstimation
import open3d as o3d

# -------------------------- USER SETTINGS ------------------------------
IMAGE_FOLDER = "../data"          # <-- put your input images here
OUTPUT_FOLDER = "glpn_pointclouds"  # where .ply files will be saved
MAX_INPUT_HEIGHT = 480           # as in the blog (controls memory)

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# --------------------------- DEVICE & MODEL ----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

feature_extractor = GLPNImageProcessor.from_pretrained("vinvino02/glpn-nyu")
model = GLPNForDepthEstimation.from_pretrained("vinvino02/glpn-nyu")
model.to(device)
model.eval()

# ------------------------ HELPER FUNCTIONS -----------------------------
def resize_to_glpn_requirements(image_pil, max_height=MAX_INPUT_HEIGHT):
    """
    Resize image so that:
    - height <= max_height
    - height and width are multiples of 32
    """
    orig_w, orig_h = image_pil.size

    new_h = max_height if orig_h > max_height else orig_h
    new_h -= (new_h % 32)

    new_w = int(new_h * orig_w / orig_h)
    diff = new_w % 32
    new_w = new_w - diff if diff < 16 else new_w + (32 - diff)

    # PIL expects (width, height)
    image_resized = image_pil.resize((new_w, new_h))
    return image_resized


def depth_from_image_glpn(image_pil):
    """
    Run GLPN on a PIL image and return:
    - depth_np: HxW depth map (float32)
    - color_cropped: PIL image cropped to match depth
    """
    # 1) Resize for model
    image_resized = resize_to_glpn_requirements(image_pil)

    # 2) Prepare input
    inputs = feature_extractor(images=image_resized, return_tensors="pt").to(device)

    # 3) Forward pass
    with torch.no_grad():
        outputs = model(**inputs)
        predicted_depth = outputs.predicted_depth  # (1,1,H,W)

    depth = predicted_depth.squeeze().cpu().numpy()  # H x W
    depth = depth * 1000.0  # scale (arbitrary, but ok)

    # 4) Remove border artifacts (same as blog)
    pad = 16
    depth_cropped = depth[pad:-pad, pad:-pad]

    # 5) Crop the RGB image correspondingly
    w_resized, h_resized = image_resized.size
    color_cropped = image_resized.crop(
        (pad, pad, w_resized - pad, h_resized - pad)
    )

    return depth_cropped.astype(np.float32), color_cropped


def depth_to_pointcloud_open3d(depth_image, color_image=None):
    """
    Convert depth (HxW) and optional color (PIL, same size) into an Open3D point cloud.
    Intrinsics are synthetic but consistent across images.
    """
    depth_np = depth_image
    h, w = depth_np.shape

    # Camera intrinsics (rough pinhole model)
    fx = fy = max(h, w)  # arbitrary but reasonable focal
    cx = w / 2.0
    cy = h / 2.0

    intrinsic = o3d.camera.PinholeCameraIntrinsic(
        width=w,
        height=h,
        fx=fx,
        fy=fy,
        cx=cx,
        cy=cy
    )

    # Open3D expects depth scale in meters; here we keep as-is
    depth_o3d = o3d.geometry.Image(depth_np)

    if color_image is not None:
        color_resized = color_image.resize((w, h))
        color_np = np.asarray(color_resized).astype(np.uint8)
    else:
        color_np = np.zeros((h, w, 3), dtype=np.uint8)

    color_o3d = o3d.geometry.Image(color_np)

    rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
        color_o3d,
        depth_o3d,
        depth_scale=1.0,       # already in arbitrary units
        depth_trunc=float(depth_np.max()) * 1.05,
        convert_rgb_to_intensity=False
    )

    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
        rgbd,
        intrinsic
    )

    # Flip to match Open3D coordinate convention (optional but standard)
    pcd.transform([[1, 0, 0, 0],
                   [0,-1, 0, 0],
                   [0, 0,-1, 0],
                   [0, 0, 0, 1]])

    return pcd

# -------------------------- MAIN LOOP ----------------------------------
image_paths = sorted(
    glob.glob(os.path.join(IMAGE_FOLDER, "*.jpg")) +
    glob.glob(os.path.join(IMAGE_FOLDER, "*.jpeg")) +
    glob.glob(os.path.join(IMAGE_FOLDER, "*.png"))
)

if not image_paths:
    raise RuntimeError(f"No images found in folder: {IMAGE_FOLDER}")

print(f"Found {len(image_paths)} images in '{IMAGE_FOLDER}'")

for idx, img_path in enumerate(image_paths):
    print(f"\n[{idx+1}/{len(image_paths)}] Processing:", os.path.basename(img_path))

    img = Image.open(img_path).convert("RGB")

    # 1) Get depth map from GLPN
    depth, color_crop = depth_from_image_glpn(img)

    # 2) Convert to point cloud
    pcd = depth_to_pointcloud_open3d(depth, color_crop)

    # 3) Save as .ply
    base = os.path.splitext(os.path.basename(img_path))[0]
    out_path = os.path.join(OUTPUT_FOLDER, f"{base}_glpn.ply")
    o3d.io.write_point_cloud(out_path, pcd)
    print("   Saved point cloud:", out_path)

print("\nDone. All point clouds saved in folder:", OUTPUT_FOLDER)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Using device: cpu


preprocessor_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

c:\Users\munee\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\munee\.cache\huggingface\hub\models--vinvino02--glpn-nyu. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/920 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/245M [00:00<?, ?B/s]

Found 33 images in '../data'

[1/33] Processing: 01.jpg


c:\Users\munee\AppData\Local\Programs\Python\Python311\Lib\site-packages\PIL\Image.py:3402: DecompressionBombWarning: Image size (108000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


   Saved point cloud: glpn_pointclouds\01_glpn.ply

[2/33] Processing: 02.jpg
   Saved point cloud: glpn_pointclouds\02_glpn.ply

[3/33] Processing: 03.jpg
   Saved point cloud: glpn_pointclouds\03_glpn.ply

[4/33] Processing: 04.jpg
   Saved point cloud: glpn_pointclouds\04_glpn.ply

[5/33] Processing: 05.jpg
   Saved point cloud: glpn_pointclouds\05_glpn.ply

[6/33] Processing: 06.jpg
   Saved point cloud: glpn_pointclouds\06_glpn.ply

[7/33] Processing: 07.jpg
   Saved point cloud: glpn_pointclouds\07_glpn.ply

[8/33] Processing: 08.jpg
   Saved point cloud: glpn_pointclouds\08_glpn.ply

[9/33] Processing: 09.jpg
   Saved point cloud: glpn_pointclouds\09_glpn.ply

[10/33] Processing: 10.jpg
   Saved point cloud: glpn_pointclouds\10_glpn.ply

[11/33] Processing: 11.jpg
   Saved point cloud: glpn_pointclouds\11_glpn.ply

[12/33] Processing: 12.jpg
   Saved point cloud: glpn_pointclouds\12_glpn.ply

[13/33] Processing: 13.jpg
   Saved point cloud: glpn_pointclouds\13_glpn.ply

[14/33] 